In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py

In [ ]:
#MAIN DIRECTORIES
mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
scratchDirectory='/homez/air673/koa_scratch/'
codeDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/CodeFiles/Project_Algorithms/Domain_Profiles/'

In [ ]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str
    
def GetData(dataDirectory, parcelDirectory):
    dataNC = xr.open_dataset(dataDirectory, decode_timedelta=True) 
    parcelNC = xr.open_dataset(parcelDirectory, decode_timedelta=True) 
    return dataNC,parcelNC

def SubsetDataVars(dataNC):
    varList = ["thflux", "qvflux", "tsk", "cape", 
               "cin", "lcl", "lfc", "th",
               "prs", "rho", "qv", "qc",
               "qr", "qi", "qs","qg", 
               "buoyancy", "uinterp", "vinterp", "winterp",]
    
    varList += ["ptb_hadv", "ptb_vadv", "ptb_hidiff", "ptb_vidiff",
                "ptb_hturb", "ptb_vturb", "ptb_mp", "ptb_rdamp", 
                "ptb_rad", "ptb_div", "ptb_diss",]
    
    varList += ["qvb_hadv", "qvb_vadv", "qvb_hidiff", "qvb_vidiff", 
                "qvb_hturb", "qvb_vturb", "qvb_mp",]
    
    varList += ["wb_hadv", "wb_vadv", "wb_hidiff", "wb_vidiff",
                "wb_hturb", "wb_vturb", "wb_pgrad", "wb_rdamp", "wb_buoy",]

    return dataNC[varList]

[dataDirectory,parcelDirectory, res,t_res,Np_str,Nz_str] = GetDataDirectories(simulationNumber=1)
[data,parcel] = GetData(dataDirectory, parcelDirectory)

In [ ]:
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

In [ ]:
import sys
path=os.path.join(mainCodeDirectory,'Functions/')
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
#LOAD VARIABLES
################################################################################
def load_vars(data_t):
    # print('MOISTURE VARIABLES'); ################################# MOISTURE VARIABLES
    rv=data_t['qv'].data;rl=data_t['qc'].data+data_t['qr'].data;
    ri=data_t['qi'].data
    
    # print('THERMODYNAMICS'); ################################# THERMODYNAMICS
    Rd=287.04
    Rv=461.5
    eps = Rd / Rv
    # Cpd=1005.7 #+-2.5
    # Cpv=1870 #+-25
    # Cpl=4190 #+-30
    # Lv0=2.501e6
    # def Lv(T): #Kirchoff's formula L_i,ii= L_i,ii0+(Cpii-Cpi)*(T-273.15)
    #     Llv=Lv0+(Cpv-Cpl)*(T-273.15) #should it be Cpl. is Cl the same?***
    #     return Llv
    
    # print('POTENTIAL TEMPERATURE'); ################################# POTENTIAL TEMPERATURE
    th=data_t['th'].data

    return th,rv,rl,ri,eps

def make_theta_v(th,rv,rl,ri,eps):
    # print('VIRTUAL POTENTIAL TEMPERATURE'); ################################# VIRTUAL POTENTIAL TEMPERATURE
    th_v = (th*(1+rv/eps))/(1+rv+rl+ri)

    # print('TEMPERATURE'); ################################# TEMPERATURE
    # T=theta*(P/p0)**(Rd/Cpd)
    
    # print('VIRTUAL TEMPERATURE'); ################################# VIRTUAL TEMPERATURE
    # eps=Rd/Rv
    # rv=data['qv'].data
    # scalar=(1+rv/eps)/(1+rv+rl+ri)
    # Tv=T*scalar
    VARs={'th_v': th_v}
    return VARs

In [ ]:
def GetOutputName():
    if res=='1km':
        outputDirectory=os.path.join(mainDirectory,'Code','OUTPUT','Variable_Calculation','CalculateMoreVariables')
        os.makedirs(outputDirectory, exist_ok=True)
    if res=='250m':
        outputDirectory=os.path.join(scratchDirectory,'OUTPUT','Variable_Calculation','CalculateMoreVariables')
        os.makedirs(outputDirectory, exist_ok=True)'
    out_file=os.path.join(outputDirectory, 'theta_v'+f'_{res}_{t_res}'+'.h5')
    return out_file
    
def initiate_array(VarNames):
    # Define array dimensions (adjust based on your data)
    t_size = len(data['time'])  # Number of timesteps
    z_size = len(data['zh'])    # Number of vertical levels
    y_size = len(data['yh'])    # Number of y-axis points
    x_size = len(data['xh'])    # Number of x-axis points

    out_file=GetOutputName()

    with h5py.File(out_file, 'a') as f:
        for var_name in VarNames:
            if var_name not in f:
                f.create_dataset(
                    var_name,
                    shape=(t_size, z_size, y_size, x_size),
                    maxshape=(None, z_size, y_size, x_size),
                    dtype='float64',
                    chunks=(1, z_size, y_size, x_size)
                )

def add_timestep_at_index(VARS, index):
    out_file=GetOutputName()
    
    with h5py.File(out_file, 'a') as f:
        for var_name, timestep_data in VARS.items():
            if var_name in f:
                f[var_name][index] = timestep_data
            else:
                raise KeyError(f"Dataset '{var_name}' does not exist in {out_file}")

In [ ]:
#RUNNING

In [ ]:
#MAKING ARRAY TO STORE theta_v
VarNames=['th_v']
initiate_array(VarNames)

#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in range(len(data['time'])):
    if np.mod(t,1)==0: print(f'Current time {t}')
    data_t=data.isel(time=t)
    [th,rv,rl,ri,eps] = load_vars(data_t)
    VARs=make_theta_v(th,rv,rl,ri,eps)
    add_timestep_at_index(VARs, t)

In [ ]:
# #READING FINAL OUTPUT
# # dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# dir2='/mnt/lustre/koa/scratch/air673/'
# file_path=dir2 + 'Variable_Calculation/' + 'theta_v'+f'_{res}_{t_res}'+'.h5'
# with h5py.File(file_path, 'a') as f:
#     # Access the existing dataset 'MSE'
#     dataset = f['th_v'][a:b]

In [ ]:
# plt.contourf(dataset[50,0])
# plt.colorbar(label='K')
# plt.title("theta_v at t = 50, z = 30 m")